In [3]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

Je pars ici sur 7 clusters : 
- luxury & fine dining
- touristic spots 
- hype / instagrammable
- fast-food, chains
- local neighborhood restaurants
- casual dining
- hidden gems

Je souhaite ici représenter chaque buisness avec un score sur 5 labels : food, inside, outside, menu et drink. Pour ca il faut définir des axes sémantiques pour chaque cluster.

In [4]:
food_axes = {
    'luxury': 'artistic gourmet plated dish with fine presentation',
    'touristic': 'famous iconic signature dish',
    'hype': 'colorful trendy instagrammable food presentation',
    'fastfood': 'fast food burger fries and soda',
    'local': 'traditional homestyle authentic cuisine',
    'casual': 'comfort food simple generous portion',
    'hidden': 'unique creative innovative dish'
}

inside_axes = {
    'luxury': 'elegant fine dining restaurant interior with white tablecloths',
    'touristic': 'busy crowded popular restaurant atmosphere',
    'hype': 'modern trendy instagram-worthy interior design',
    'fastfood': 'fast food chain restaurant counter',
    'local': 'authentic neighborhood restaurant cozy family atmosphere',
    'casual': 'relaxed comfortable dining space',
    'hidden': 'intimate small restaurant with unique character'
}

outside_axes = {
    'luxury': 'upscale restaurant elegant sophisticated facade',
    'touristic': 'tourist destination restaurant',
    'hype': 'stylish modern trendy storefront',
    'fastfood': 'recognizable chain restaurant brand exterior',
    'local': 'traditional neighborhood local restaurant',
    'casual': 'welcoming informal restaurant entrance',
    'hidden': 'discreet unassuming small restaurant'
}

drink_axes = {
    'luxury': 'craft cocktail sophisticated wine presentation',
    'touristic': 'popular signature drink',
    'hype': 'colorful photogenic cocktail or beverage',
    'fastfood': 'soda fountain soft drink',
    'local': 'traditional local beverage or beer',
    'casual': 'casual drinks',
    'hidden': 'unique specialty drink'
}

menu_axes = {
    'luxury': 'refined curated short menu',
    'touristic': 'extensive varied tourist-friendly menu with big pictures',
    'hype': 'modern trendy menu design',
    'fastfood': 'standardized chain menu board',
    'local': 'traditional minimalist local menu, can be handwritten',
    'casual': 'classic diner menu',
    'hidden': 'small specialized unique menu'
}